# Task 5.2: A Guide to Migrating from Qiskit Primitives V1 to V2

This notebook provides a practical, hands-on guide for updating your code from Qiskit's V1 Primitives to the modern V2 Primitives (`EstimatorV2`, `SamplerV2`).

**Note:** V1 Primitives are deprecated in recent Qiskit versions. This guide shows V1 code patterns as **commented examples** alongside **working V2 code** you can run locally with `AerSimulator`.

## Setup: Imports and Backend

First, we'll import everything we need for V2 primitives. All examples will run on a local `AerSimulator`.

In [ ]:
# General Qiskit tools
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
import numpy as np

# The local simulator backend we will use for all examples
from qiskit_aer import AerSimulator

# --- V2 Primitives (Current/Recommended) ---
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit_ibm_runtime import SamplerV2 as Sampler

# Tool to pretty-print options dictionaries
from dataclasses import asdict

# Define a local backend instance to use throughout
local_backend = AerSimulator()

print("Imports complete and local backend is ready.")

In [ ]:
# Helper function to create a Bell state circuit
def create_bell_circuit():
    """Create a Bell state (maximally entangled state) circuit."""
    qc = QuantumCircuit(2)
    qc.h(0)  # Apply Hadamard to qubit 0
    qc.cx(0, 1)  # Apply CNOT with control=0, target=1
    return qc

# Test it
bell = create_bell_circuit()
print(bell)

## 1. The `run()` Method: Circuits, Observables, and Parameters

The most significant change from V1 to V2 is the structure of the `run()` method's inputs.

### Key Differences:
- **V1:** Took separate arguments for `circuits`, `observables`, and `parameter_values`
- **V2:** Takes a single list of **PUBs (Primitive Unified Blocs)** - tuples that group a circuit with its observables and parameters

### Estimator: V1 → V2 Migration

Let's see how to migrate a simple Bell circuit expectation value calculation.

In [ ]:
print("=" * 60)
print("V1 CODE (Deprecated - shown for reference):")
print("=" * 60)
print("""
# V1 Estimator - OLD WAY (Don't use this anymore)
from qiskit_ibm_runtime import Estimator

circuit = create_bell_circuit()
observable = SparsePauliOp("ZZ")

estimator = Estimator(backend=local_backend)
# V1 used separate keyword arguments
job = estimator.run(
    circuits=[circuit],
    observables=[observable]
)
result = job.result()
print(f"Expectation Value: {result.values[0]}")
""")

print("\n" + "=" * 60)
print("V2 CODE (Current - Working Example):")
print("=" * 60)

In [ ]:
# V2 Estimator - NEW WAY (Use this!)
from qiskit_ibm_runtime import EstimatorV2 as Estimator

circuit = create_bell_circuit()
observable = SparsePauliOp("ZZ")

estimator = Estimator(mode=local_backend)

# V2 uses PUBs: a list of tuples (circuit, observable, [optional params])
job = estimator.run(pubs=[(circuit, observable)])
result = job.result()

# V2 results are indexed by PUB, and data is in .data attribute
pub_result = result[0]
print(f"V2 Job ID: {job.job_id()}")
print(f"V2 Expectation Value: {pub_result.data.evs}")
print(f"V2 Standard Deviation: {pub_result.data.stds}")

### Sampler: V1 → V2 Migration

The pattern is the same for the Sampler.

In [ ]:
print("=" * 60)
print("V1 CODE (Deprecated - shown for reference):")
print("=" * 60)
print("""
# V1 Sampler - OLD WAY
from qiskit_ibm_runtime import Sampler

circuit = create_bell_circuit()
circuit.measure_all()

sampler = Sampler(backend=local_backend)
# V1 used separate keyword argument
job = sampler.run(circuits=[circuit])
result = job.result()
print(f"Quasi-distribution: {result.quasi_dists[0]}")
""")

print("\n" + "=" * 60)
print("V2 CODE (Current - Working Example):")
print("=" * 60)

In [ ]:
# V2 Sampler - NEW WAY (Use this!)
from qiskit_ibm_runtime import SamplerV2 as Sampler

circuit = create_bell_circuit()
circuit.measure_all()

sampler = Sampler(mode=local_backend)

# V2 uses PUBs: a list of tuples (circuit, [optional params], [optional shots])
# Single circuit with no params - note the comma to make it a tuple
job = sampler.run(pubs=[(circuit,)])
result = job.result()

# V2 results are indexed by PUB
pub_result = result[0]
print(f"V2 Job ID: {job.job_id()}")
print(f"V2 Bitstring counts: {pub_result.data.meas.get_counts()}")
print(f"V2 First 5 bitstrings: {pub_result.data.meas.get_bitstrings()[:5]}")

## 2. Handling Multiple Jobs with Different Parameters

The V2 PUB system makes it much easier to run varied jobs. You can specify different shots, parameters, or observables for each circuit.

### Example: Two Circuits with Different Shot Counts

In [ ]:
# Create two different circuits
circuit1 = create_bell_circuit()
circuit1.measure_all()

circuit2 = create_bell_circuit()
circuit2.ry(np.pi / 4, 0)  # Add extra rotation
circuit2.measure_all()

# V2 PUB format for Sampler: (circuit, parameter_values, shots)
# We can specify different shots for each circuit!
pub1 = (circuit1, None, 100)   # Circuit 1 with 100 shots
pub2 = (circuit2, None, 500)   # Circuit 2 with 500 shots

sampler = Sampler(mode=local_backend)
job = sampler.run(pubs=[pub1, pub2])
result = job.result()

# Results are indexed corresponding to the input PUBs
print(f"Result for Circuit 1 (shots={result[0].metadata['shots']}):\n  {result[0].data.meas.get_counts()}")
print(f"\nResult for Circuit 2 (shots={result[1].metadata['shots']}):\n  {result[1].data.meas.get_counts()}")

### Example: One Circuit with Multiple Observables

V2 makes it easy to measure multiple observables on the same circuit.

In [ ]:
circuit = create_bell_circuit()

# Define multiple observables to measure
obs1 = SparsePauliOp("ZZ")
obs2 = SparsePauliOp("XX")
obs3 = SparsePauliOp("YY")

# V2 PUB format: (circuit, [list of observables])
estimator = Estimator(mode=local_backend)
job = estimator.run(pubs=[(circuit, [obs1, obs2, obs3])])
result = job.result()

pub_result = result[0]
print("Expectation values for multiple observables:")
print(f"  ZZ: {pub_result.data.evs[0]}")
print(f"  XX: {pub_result.data.evs[1]}")
print(f"  YY: {pub_result.data.evs[2]}")

## 3. Accessing Results (Output Structure Changes)

The result structure has changed significantly:

### V1 Results:
```python
# Estimator V1
result.values[0]           # Array of expectation values
result.metadata[0]         # Metadata for job 0

# Sampler V1  
result.quasi_dists[0]      # Quasi-probability distribution
```

### V2 Results:
```python
# Estimator V2
result[0].data.evs         # Expectation values for PUB 0
result[0].data.stds        # Standard deviations for PUB 0
result[0].metadata         # Metadata for PUB 0

# Sampler V2
result[0].data.meas.get_counts()      # Measurement counts
result[0].data.meas.get_bitstrings()  # Bitstring array
```

### Example: Converting V2 Sampler Output to V1-Style QuasiDistribution

In [ ]:
# Run a V2 Sampler job
circuit = create_bell_circuit()
circuit.measure_all()

sampler = Sampler(mode=local_backend)
job = sampler.run([(circuit,)], shots=1024)
result = job.result()

# V2 result structure
pub_result = result[0]
shots = pub_result.metadata['shots']
counts = pub_result.data.meas.get_counts()

print(f"V2 Counts (dict of bitstrings): {counts}")
print(f"Total shots: {shots}")

# Convert to V1-style quasi-distribution (dict with integer keys)
v1_quasi_dist = {int(bitstring, 2): count/shots for bitstring, count in counts.items()}
print(f"\nV1-style QuasiDistribution (for migration): {v1_quasi_dist}")

## 4. Setting Options

Options configuration has been streamlined in V2.

### V1 Approach (Deprecated):
```python
from qiskit_ibm_runtime import Options

options = Options()
options.resilience_level = 1
options.execution.shots = 4000
estimator = Estimator(backend=backend, options=options)

# Or update later:
estimator.set_options(resilience_level=2)
```

### V2 Approach (Current):
The primitive has a built-in `.options` attribute you can set directly.

In [ ]:
# Method 1: Pass options as a dictionary during initialization
estimator = Estimator(
    mode=local_backend,
    options={"resilience_level": 1, "default_shots": 2000}
)
print(f"Resilience level: {estimator.options.resilience_level}")
print(f"Default shots: {estimator.options.default_shots}")

In [ ]:
# Method 2: Set attributes directly (enables auto-complete in IDEs)
estimator = Estimator(mode=local_backend)
estimator.options.resilience_level = 1
estimator.options.default_shots = 4000

print(f"Resilience level: {estimator.options.resilience_level}")
print(f"Default shots: {estimator.options.default_shots}")

In [ ]:
# Method 3: Use update() for bulk updates
estimator = Estimator(mode=local_backend)
estimator.options.update(
    resilience_level=0,
    default_shots=1024
)

print(f"Resilience level: {estimator.options.resilience_level}")
print(f"Default shots: {estimator.options.default_shots}")

# View all options as a dictionary
print(f"\nAll options:\n{asdict(estimator.options)}")

## 5. Summary: Migration Checklist

### Steps to Migrate to EstimatorV2:

1. **Update imports:**
   ```python
   # Old: from qiskit_ibm_runtime import Estimator
   # New:
   from qiskit_ibm_runtime import EstimatorV2 as Estimator
   ```

2. **Remove the Options import** (no longer needed)

3. **Change backend parameter:**
   ```python
   # Old: estimator = Estimator(backend=backend)
   # New:
   estimator = Estimator(mode=backend)
   ```

4. **Update options setting:**
   ```python
   # Old: estimator.set_options(resilience_level=1)
   # New:
   estimator.options.resilience_level = 1
   ```

5. **Group inputs into PUBs:**
   ```python
   # Old: job = estimator.run(circuits=[circuit], observables=[obs])
   # New:
   job = estimator.run(pubs=[(circuit, obs)])
   ```

6. **Update result access:**
   ```python
   # Old: result.values[0]
   # New:
   result[0].data.evs
   ```

### Steps to Migrate to SamplerV2:

1. **Update imports:**
   ```python
   from qiskit_ibm_runtime import SamplerV2 as Sampler
   ```

2. **Change backend parameter:**
   ```python
   sampler = Sampler(mode=backend)
   ```

3. **Update options:**
   ```python
   sampler.options.default_shots = 4096
   ```

4. **Group inputs into PUBs:**
   ```python
   # Old: job = sampler.run(circuits=[circuit])
   # New:
   job = sampler.run(pubs=[(circuit,)])  # Note the comma!
   ```

5. **Update result access:**
   ```python
   # Old: result.quasi_dists[0]
   # New:
   result[0].data.meas.get_counts()
   # or
   result[0].data.meas.get_bitstrings()
   ```

## Conclusion

The V2 Primitives offer:
- **More flexibility** with the PUB system
- **Cleaner API** with built-in options
- **Better support** for varied workloads (different shots, observables, parameters per circuit)
- **Improved performance** and error mitigation capabilities

All code in this notebook runs locally without requiring cloud access, making it easy to practice and test your migrations!